# Text Generation

사전학습된 CLM(Causal Language Model)을 이용하여 자연어 문장을 생성하는 방법에 대해 살펴 보겠습니다.  
GPT-2 XL은 GPT-2의 1.5B 파라미터 버전으로 트랜스포머 기반의 CLM 입니다.  
Greedy Search Decoding, Beam Search Decoding, Random Sampling, Top-K/Top-P Sampling 방법을 실습합니다.  
Ref. Natural Language Processing with Transformers

### 0. Setup

In [ ]:
!pip install --user transformers

In [ ]:
# MLP Suwon 설정 필요
import os

os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'

proxies = {
'http': '75.17.107.42:8080',
'https': '75.17.107.42:8080'
}

In [ ]:
# MLP Suwon 설정 필요
import ssl

if hasattr(ssl, '_create_unverified_context'):
   ssl._create_default_https_context = ssl._create_unverified_context

- **gpt-2 xl**: 48-layer, 1600-hidden, 25-heads, 1,558M parameters, OpenAI's XL-sized GPT-2 English model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
# 다음 코드를 완성하세요!! (사전학습 모델에 사용된 Tokenizer 가져오기)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

### 1. Greedy Search Decoding

In [ ]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []

n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # 첫번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다.
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # 가장 높은 확률의 토큰을 저장합니다.
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob: .2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # 예측한 다음 토큰을 입력에 추가합니다.    
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

In [ ]:
pd.DataFrame(iterations)

In [ ]:
max_length = 12

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
# 다음 코드를 완성하세요!! (입력에 대한 Greedy Search: input_ids, max_length, do_sample 설정)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

### 2. Beam Search Decoding

- **log_probs_from_logits()** 함수: 하나의 토큰에 대한 로그 확률을 제공합니다.
- **sequence_logprob()** 함수: 시퀀스에 대한 전체 로그 확률값을 계산합니다.

In [ ]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [ ]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

빔서치 디코딩은 확률이 가장 높은 상위 num_beam 갯수 만큼의 다음 토큰 시퀀스를 추적합니다. 

In [ ]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

In [ ]:
# 다음 코드를 완성하세요!! (입력에 대한 Beam Search: input_ids, max_length, num_beam, do_sample 설정)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))

print(tokenizer.decode(output_beam[0]))
print(f"\nLog Probability: {logp:.2f}")

- **no_repeat_ngram_size**: 텍스트가 반복되는 문제를 해결하기 위하여 n-gram penalty를 부과할 수도 있습니다. 

In [ ]:
# 다음 코드를 완성하세요!! (이전 Beam Search 결과에 no_repeat_ngram_size 설정)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))

print(tokenizer.decode(output_beam[0]))
print(f"\nLog Probability: {logp:.2f}")

### 3. Random Sampling

Random Sampling은 각 타임스텝 내 모델이 출력한 전체 어휘사전에 대한 확률분포에서 랜덤하게 샘플링하는 방법입니다.  
- **temperature**: 소프트맥스 함수를 적용하기 전에 로짓의 스케일을 조정하는 Temperature 파라미터를 추가하면 출력의 다양성을 제어할 수 있습니다.  
T << 1 일때 낮은 확률의 토큰들을 억제하며, T >> 1 일때는 분포가 평평해져서 각 토큰의 확률들이 동일해집니다.

In [ ]:
# 다음 코드를 완성하세요!! (Random Sampleing 방법을 통한 문장 생성: input_ids, max_length, do_sample, temperature 설정)
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)

print(tokenizer.decode(output_temp[0]))

In [ ]:
# 다음 코드를 완성하세요!! (Temperature를 변경하여 문장 생성: input_ids, max_length, do_sample, temperature 설정)
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)

print(tokenizer.decode(output_temp[0]))

### 4. Top-K and Top-P Sampling

두 방법 모두 샘플링에 사용할 토큰의 갯수를 줄인다는 개념에 기초하고 있습니다.  

- Top-K 샘플링: 확률이 가장 높은 K개 토큰에서만 샘플링하고 확률이 낮은 토큰을 제외함으로써,
확률 분포의 롱테일을 잘라내고 확률이 가장 높은 토큰에서만 샘플링하는 방법입니다. 

In [ ]:
# 다음 코드를 완성하세요!! (Top-K 샘플링을 적용하여 문장 생성: input_ids, max_length, do_sample, top_k 설정)
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)

print(tokenizer.decode(output_topk[0]))

- Top-P 샘플링은 고정된 컷오프 값을 사용하지 않고, 어디서 컷오프할 것인지 확률질량(Probability Mass) 조건을 지정합니다.  
모든 토큰을 확률에 따라 내림차순으로 정렬하고, 누적확률값에 도달할 때까지 토큰들을 하나씩 추가하게 됩니다.

In [ ]:
# 다음 코드를 완성하세요!! (Top-P 샘플링을 적용하여 문장 생성: input_ids, max_length, do_sample, top_k 설정)
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.90)

print(tokenizer.decode(output_topp[0]))